In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

c:\Users\Admin\anaconda3\envs\jupyterlab\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [2]:
# Load GPT-Neo 1.3B model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"  # Model name for GPT-Neo 1.3B
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

# Set pad_token to eos_token if it's not already defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token



In [3]:
def extract_calories_with_gpt_neo(meal_description):
    # Create the prompt for the model
    prompt = f"""
    Count words in the input:
    "{meal_description}"

    Provide as a number
    """

    
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, add_special_tokens=True).to(model.device)
    
    # Ensure the attention mask and pad_token_id are set
    attention_mask = inputs["attention_mask"]
    pad_token_id = tokenizer.pad_token_id

    # Generate the output (response) from the model
    outputs = model.generate(
        inputs["input_ids"], 
        max_length=60,  # Limit the length to prevent excessive text generation
        temperature=0.2,  # Lower temperature for more deterministic outputs
        num_return_sequences=1,  # Generate a single response
        do_sample=False,  # Disable sampling to make the output deterministic
        attention_mask=attention_mask,  # Pass the attention mask
        pad_token_id=pad_token_id  # Pass the pad token id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response
    
    # # Debug: Print the full response to inspect it
    # print("Response from GPT-Neo 1.3B:")
    # print(response)

    # # Extract the calorie estimate (the last number in the response)
    # try:
    #     calorie_estimate = int([word for word in response.split() if word.isdigit()][-1])
    #     return calorie_estimate
    # except ValueError:
    #     return "Error parsing response"


In [5]:

# Sample meal descriptions
meal_texts = [
    "toast with butter and a cup of coffee with milk.",
    "bowl of chicken salad with dressing.",
    "spaghetti bolognese with garlic bread.",
]

# Extract calorie estimates
for meal in meal_texts:
    estimated_calories = extract_calories_with_gpt_neo(meal)
    print(f"Meal: {meal}\nEstimated Calories: {estimated_calories}\n")


Meal: toast with butter and a cup of coffee with milk.
Estimated Calories: 
    Count words in the input:
    "toast with butter and a cup of coffee with milk."

    Provide as a number
        the number of words in the input:
        "

Meal: bowl of chicken salad with dressing.
Estimated Calories: 
    Count words in the input:
    "bowl of chicken salad with dressing."

    Provide as a number
                             

Meal: spaghetti bolognese with garlic bread.
Estimated Calories: 
    Count words in the input:
    "spaghetti bolognese with garlic bread."

    Provide as a number
                          



In [56]:
extract_calories_with_gpt_neo('glass of milk')

'\n    Estimate the calorie content of the following meal:\n    "glass of milk"\n\n    Provide only the calorie estimate as a number, and nothing else. Example: 400\n      calories.\n\n        '

In [51]:
meal

'spaghetti bolognese with garlic bread.'

In [52]:
estimated_calories

'\nEstimate the calorie content of the following meal:\n"For dinner, I had spaghetti bolognese with garlic bread, totaling about 700 calories."\n\nProvide only the calorie estimate as a number.\n\nProvide only the'

In [62]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Load LLaMA 2 model from Hugging Face
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Sample meal description
# meal_description = "A glass of milk."

# # Create a prompt
# prompt = f"""
# Estimate the calorie content of the following meal:
# "{meal_description}"

# Provide only the calorie estimate as a number, for example: 150.
# """

# # Tokenize the input prompt
# inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

# # Generate the output (response) from the model
# outputs = model.generate(inputs["input_ids"], max_length=20, do_sample=False)

# # Decode and print the response
# response = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(response)
